In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from utils import  flatten,post_prob
from pca import PCA
from rda import RDA

In [2]:
main_path = '.\DevanagariHandwrittenCharacterDataset'
os.listdir(main_path)

['Test', 'Train']

In [3]:
train_path = os.path.join(main_path,'Train')
test_path = os.path.join(main_path,'Test')

In [4]:
train_folder = os.listdir(train_path)
test_folder = os.listdir(test_path)

In [5]:
train_image_folder_path = list(map(lambda x : os.path.join(train_path,x),train_folder))
test_image_folder_path = list(map(lambda x : os.path.join(test_path,x),test_folder))

In [6]:
train_image_path = []
for i in train_image_folder_path:
    for j in os.listdir(i):
        train_image_path.append(os.path.join(i,j))

In [7]:
test_image_path = []
for i in test_image_folder_path:
    for j in os.listdir(i):
        test_image_path.append(os.path.join(i,j))

Flattening the images so as to convert them for proper input size for our model.

In [8]:
train_images = np.concatenate(flatten(train_image_path))

In [9]:
test_images = np.concatenate(flatten(test_image_path))

In [10]:
unique_labels = np.arange(0,len(train_image_folder_path))

In [11]:
class_labels = []
class_labels.extend(map(lambda x : x*np.ones((1700,1)),unique_labels))

In [12]:
class_labels = np.concatenate(class_labels)

## Dataset

In [13]:
train_df = pd.DataFrame(train_images)

In [14]:
train_df['labels'] = class_labels

In [15]:
test_df = pd.DataFrame(test_images)

In [16]:
test_labels = np.concatenate(list(map(lambda x : x * np.ones((300,1)),unique_labels)))

In [17]:
test_df['label'] = test_labels

## Applying PCA on our dataset.

**PCA** : Principal component analysis (PCA) is a technique for reducing the dimensionality of such datasets, increasing interpretability but at the same time minimizing information loss. It does so by creating new uncorrelated variables that successively maximize variance.


![](https://miro.medium.com/max/1200/1*ba0XpZtJrgh7UpzWcIgZ1Q.jpeg)

In [20]:
eig_mat = PCA(np.array(train_df.iloc[:,0:1024].cov()))

In [21]:
eig_mat.shape

(1024, 239)

In [22]:
reduced_data = np.matmul(np.array(train_df.iloc[:,:1024]),eig_mat)

In [23]:
reduced_data.shape

(78200, 239)

Calculating class covariances for each class

In [24]:
class_covarainces = list(map(lambda x : train_df[train_df['labels'] == x].iloc[:,:1024].cov() , unique_labels))

## Applying RDA
**RDA** : Regularized discriminant analysis uses the same general setup as LDA and QDA but estimates the covariance in a new way, which combines the covariance of QDA using (Σk) and with the covariance of LDA (Σ) using tuning parameter λ.

$$ \sigma_k(\lambda) = (1 - \lambda)\sigma_k + \lambda \sigma $$


$$\sigma_k(\lambda,\gamma) = (1 - \gamma) \sigma_k(\lambda) + \gamma \frac1p tr(\sigma_k(\lambda))I$$
Both γ and λ can be thought of as mixing parameters, as they both take values between 0 and 1. For the four extremes of γ and λ, the covariance structure reduces to special cases:
- (γ=0,λ=0): QDA - individual covariance for each group.
- (γ=0,λ=1): LDA - a common covariance matrix.
- (γ=1,λ=0): Conditional independent variables - similar to Naive Bayes, but variable variances within group (main diagonal elements) are all equal.
- (γ=1,λ=1): Classification using euclidean distance - as in previous case, but variances are the same for all groups. Objects are assigned to group with nearest mean.

In [51]:
regularised_matrix = RDA(class_covarainces,train_df,train_folder,unique_labels)

In [52]:
np.linalg.det(regularised_matrix)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [53]:
reduced_data.shape

(78200, 239)

### Calculating class covariance and mean vector for each class so that we can use calculate to **Posterior Probability** so as to perform classification.

In [54]:
classes_cov_mats_reduced = []

classes_cov_mats_reduced.extend(map(lambda x: np.cov(reduced_data[(1700*x):(1700*(x+1)),:],rowvar=False),unique_labels))

In [56]:
class_mean_vecs = []

class_mean_vecs.extend(map(lambda x: np.mean(reduced_data[1700*x:1700*(x+1),:],axis=0),np.arange(0,len(train_folder))))

#### Applying RDA on reduced data obtained after applying PCA

In [58]:
pca_reduced_rda_matrices = RDA(classes_cov_mats_reduced,train_df,train_folder,unique_labels)

In [59]:
pca_reduced_test_data = np.matmul(test_df.iloc[:,:1024],eig_mat)

In [60]:
pca_reduced_test_data.shape

(13800, 239)

In [61]:
actual_classes = np.concatenate(test_labels)

In [62]:
actual_classes = actual_classes.reshape(13800,1)

In [63]:
post_probs = []

post_probs.extend(map(lambda x: post_prob(class_mean_vecs[x],pca_reduced_rda_matrices[x],pca_reduced_test_data),np.arange(0,len(train_folder))))

In [64]:
posterior_probabilities = np.concatenate(post_probs,axis=1)

In [65]:
posterior_probabilities.shape

(13800, 46)

In [66]:
predicted_classes = np.argmax(posterior_probabilities,axis=1)

In [67]:
predicted_classes = predicted_classes.reshape(13800,1)

In [68]:
actual_classes.shape

(13800, 1)

In [69]:
correct_count = np.count_nonzero(np.equal(predicted_classes,actual_classes))

In [75]:
correct_count

12606

In [76]:
accuracy = (correct_count/actual_classes.shape[0])*100

In [78]:
accuracy

91.34782608695652